<h1> DattoRMM - Devices Audit Info - Data Ingestion - MongoDB</h1>

# Import Modules

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import json
import re


# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

# import configparser for env secrets
from configparser import ConfigParser
config = ConfigParser()
config.read('d:/git/example_infrastructure_data_dev/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

# DattoRMM - Pull JSON from API

## Create Auth Token

### Create DataFrame via API Call Iteration


In [ ]:
# DattoRMM DataFrame - import and assign secrets from env.ini
base_uri = config['dattormm']['base_uri']
api_key = config['dattormm']['api_key']
api_secret = config['dattormm']['api_secret']
# call token api url
token_uri = config['dattormm']['token_uri']

In [ ]:
# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = api_key
data['password'] = api_secret

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']
## Create Devices DataFrame
# request content response
request_url = f'{base_uri}/account/devices'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)
print(c_dict['pageDetails']['nextPageUrl'])

# iterate and combine remaining pages
df_devices = pd.DataFrame(c_dict['devices'])
while c_dict['pageDetails']['nextPageUrl']:
    next_page = c_dict['pageDetails']['nextPageUrl']
    resp = requests.get(next_page, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)
    print(c_dict['pageDetails']['nextPageUrl'])
    df_current_page = pd.DataFrame(c_dict['devices'])
    df_devices = pd.concat([df_devices, df_current_page], ignore_index=False)

# Data Shaping

## Create New Columns from Dictionary Columns

### Set Index to device UID

In [ ]:
df_devices.set_index('uid',inplace=True)

# MongoDB
## Import Modules and env Variables

In [ ]:
username = config['mongodb']['username']
password = config['mongodb']['password']
connection_ip = config['mongodb']['connection_ip']
database = 'seed_data'

# import bson for object encoding
import bson

# Provide the mongodb atlas url to connect python to mongodb using pymongo
#CONNECTION_STRING = f"mongodb://{username}:{password}@{connection_ip}/{database}"
CONNECTION_STRING = 'mongodb://localhost:27017'


# Create a connection using MongoClient. You can import MongoClient or use pymongo.MongoClient
from pymongo import MongoClient
client = MongoClient(CONNECTION_STRING)


db = client['datto_rmm']
collection = db['seed_data']


In [ ]:
df_audit_info = df_devices.reset_index()
dt_list = list(df_audit_info.dtypes[df_audit_info.dtypes == 'datetime64[ns]'].index)
df_audit_info.fillna(pd.NA,inplace=True)
df_audit_info = df_audit_info[['uid']]

# Create Construct Expanded DataFrame

## Create DataFrame of all UID's found in DattoRMM for Comparison to MongoDB

In [ ]:
for index, row in df_audit_info.iterrows():
    row_info = row.to_dict()
    deviceUid = row_info['uid']

## Create two lists based on found MongoDB entries matching DattoUID

In [ ]:
found_entries = []
missing_entries = []
for index, row in df_audit_info.iterrows():
    row_info = row.to_dict()
    deviceUid = row_info['uid']
    try:
        result = collection.find_one({'uid':deviceUid})
        #print(f"_id found: {result['_id']}")
        found_entries.append({'uid':deviceUid,'match_id':result['_id']})
    except:
        missing_entries.append(deviceUid)
        print(f'_id missing: {deviceUid}')
    #for k,v in entry.items():
     #   print(k,v)

## Using Found _id's find matching uid device audit lists via DattoAPI and insert into Mongo as iterating

In [ ]:
for entry in found_entries:


    # request content response
    request_url = f"https://concord-api.centrastage.net/api/v2/audit/device/{entry['uid']}"
    #request_url = f'{base_uri}/job'
    # construct header
    headers = CaseInsensitiveDict()
    headers['Authorization'] = f'Bearer {access_token}'
    headers['Content-Type'] = 'application/json'

    # construct req body
    data = CaseInsensitiveDict()
    data['jobComponent'] = '[REDACTED_UUID] '

    resp = requests.get(request_url, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)
    device_info_json = c_dict


    #using data from DattoRMM API > begin update_one records
    result = collection.find_one({'uid':entry['uid']})
    print(f"_id found: {result['_id']}")
    match_id = result['_id']
    for k,v in device_info_json.items():
        # drop all uid or null values to reduce insertions of empty objects
        if (k == 'match_id') | (k == 'uid') | (v == None) | (v == []):
            print(f'skipping: {k,v}')
            continue
        else:
            try:
                collection.update_one({'_id':match_id}, {'$set':{f"{str(k)}":v}},upsert=True)
                print(f"Successfully inserted new {k} for {entry['uid']}")
            except Exception as error:
                print(f"Unable to insert new {k} for {entry['uid']}")
                print(error)


print('*'*120)
print('End of Import!')

# Construct Jobs DataFrame

# Construct Jobs DataFrame